<a href="https://colab.research.google.com/github/issacridhin/S5_Predictive-Analysis/blob/main/2348546_Ridhin_PA_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

In [10]:

df = pd.read_csv('MARUTI.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)


In [11]:
df.head(6)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2003-07-09,MARUTI,EQ,125.00,164.90,170.40,155.00,164.0,164.30,165.95,35164283,5.835528e+14,NaN,8537695.0,0.2428
2003-07-10,MARUTI,EQ,164.30,167.00,168.70,164.50,167.0,167.00,166.74,10464179,1.744820e+14,NaN,4363947.0,0.4170
2003-07-11,MARUTI,EQ,167.00,167.75,174.85,166.25,173.6,173.35,172.45,11740117,2.024622e+14,NaN,3014852.0,0.2568
2003-07-14,MARUTI,EQ,173.35,174.25,179.25,174.25,178.6,177.95,177.91,5982324,1.064313e+14,NaN,1949217.0,0.3258
2003-07-15,MARUTI,EQ,177.95,200.00,200.00,173.00,176.3,176.20,176.88,6173689,1.092001e+14,NaN,1307694.0,0.2118
2003-07-16,MARUTI,EQ,176.20,176.45,179.10,175.35,176.9,177.10,177.59,3759085,6.675695e+13,NaN,1155442.0,0.3074


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4427 entries, 2003-07-09 to 2021-04-30
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Symbol              4427 non-null   object 
 1   Series              4427 non-null   object 
 2   Prev Close          4427 non-null   float64
 3   Open                4427 non-null   float64
 4   High                4427 non-null   float64
 5   Low                 4427 non-null   float64
 6   Last                4427 non-null   float64
 7   Close               4427 non-null   float64
 8   VWAP                4427 non-null   float64
 9   Volume              4427 non-null   int64  
 10  Turnover            4427 non-null   float64
 11  Trades              2456 non-null   float64
 12  Deliverable Volume  4426 non-null   float64
 13  %Deliverble         4426 non-null   float64
dtypes: float64(11), int64(1), object(2)
memory usage: 518.8+ KB


In [14]:
df.isnull().sum()

,0
Symbol,0
Series,0
Prev Close,0
Open,0
High,0
Low,0
Last,0
Close,0
VWAP,0
Volume,0


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define a custom function for each preprocessing step
def smooth_close(series, window=7):
    return series.rolling(window=window).mean()

def detect_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return np.where((series < lower_bound) | (series > upper_bound), np.nan, series)

def fill_missing_values(series, method='ffill'):
    return series.fillna(method=method)

# Define the pipeline steps
preprocessing_pipeline = Pipeline([
    ('smooth_close', ('Close', lambda df: smooth_close(df['Close']))),  # Smooth 'Close' column
    ('outlier_removal_volume', ('Volume', lambda df: detect_outliers(df['Volume']))),  # Remove outliers in 'Volume'
    ('fill_missing_trades', ('Trades', lambda df: fill_missing_values(df['Trades'], method='ffill'))),  # Fill missing 'Trades' column
    ('scaling', StandardScaler())  # Scale the data (for all columns)
])

# Apply pipeline transformations
df['Close_smooth'] = smooth_close(df['Close'])          # Apply smoothing
df['Volume_clean'] = detect_outliers(df['Volume'])      # Remove outliers in Volume
df['Trades_filled'] = fill_missing_values(df['Trades']) # Handle missing Trades

# Apply scaling (Z-score normalization)
scaler = StandardScaler()
df[['Open', 'High', 'Low', 'Close_smooth', 'Volume_clean']] = scaler.fit_transform(
    df[['Open', 'High', 'Low', 'Close_smooth', 'Volume_clean']]
)

# Display final dataset
print(df[['Open', 'High', 'Low', 'Close_smooth', 'Volume_clean', 'Trades_filled']].head(10))

                Open      High       Low  Close_smooth  Volume_clean  \
Date                                                                   
2003-07-09 -1.006463 -1.008248 -1.007009           NaN           NaN   
2003-07-10 -1.005698 -1.008862 -1.003511           NaN           NaN   
2003-07-11 -1.005425 -1.006641 -1.002866           NaN           NaN   
2003-07-14 -1.003057 -1.005053 -0.999920           NaN           NaN   
2003-07-15 -0.993677 -0.997561 -1.000380           NaN           NaN   
2003-07-16 -1.002256 -1.005107 -0.999514           NaN           NaN   
2003-07-17 -1.001873 -1.005504 -1.001245     -1.004801           NaN   
2003-07-18 -1.006426 -1.007706 -1.005168     -1.004640           NaN   
2003-07-21 -1.004969 -1.008609 -1.004045     -1.004749           NaN   
2003-07-22 -1.006791 -1.008699 -1.004394     -1.005067           NaN   

            Trades_filled  
Date                       
2003-07-09            NaN  
2003-07-10            NaN  
2003-07-11            N

<ipython-input-15-c370c201e52a>:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return series.fillna(method=method)
